In [1]:
# =========================
# Cell 1: Setup and Imports
# =========================
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time

torch.set_default_dtype(torch.float64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [2]:
# =========================
# Cell 2: Network Definition
# =========================
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2, 100), nn.Tanh(),
            nn.Linear(100, 100), nn.Tanh(),
            nn.Linear(100, 100), nn.Tanh(),
            nn.Linear(100, 100), nn.Tanh(),
            nn.Linear(100, 1)
        )
    def forward(self, x):
        return self.layers(x)

model = Net().to(device)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)
model.apply(init_weights)

Net(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Tanh()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): Tanh()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): Tanh()
    (8): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [3]:
# =============================
# Cell 3: Exact Solution & RHS
# =============================
def exact_solution(x):
    return (1/(2*np.pi**2)) * torch.sin(np.pi*x[:,0]) * torch.sin(np.pi*x[:,1])

def rhs_f(x):
    return (np.pi**4 / 2) * torch.sin(np.pi*x[:,0]) * torch.sin(np.pi*x[:,1])

def dirichlet_bc(x):
    return exact_solution(x)

def neumann_bc(x, n):
    # u_exact = (1/2pi^2) * sin(pi*x) * sin(pi*y)
    # du/dx = (1/2pi) * cos(pi*x) * sin(pi*y)
    # du/dy = (1/2pi) * sin(pi*x) * cos(pi*y)
    
    du_dx = (1/(2*np.pi)) * torch.cos(np.pi*x[:,0]) * torch.sin(np.pi*x[:,1])
    du_dy = (1/(2*np.pi)) * torch.sin(np.pi*x[:,0]) * torch.cos(np.pi*x[:,1])
    
    # Normal derivative = dot product of gradient and normal
    # n has shape (N, 2), du_dx/dy have shape (N,)
    du_dn = du_dx * n[:,0] + du_dy * n[:,1]
    return du_dn

In [4]:
# ==============================================
# Cell 4: Collocation and Boundary Point Sampler
# ==============================================
N_int, N_bd = 8000, 4000
def get_interior(N):
    return torch.rand(N, 2, device=device)

def get_boundary(M):
    grid = torch.linspace(0, 1, M//4, device=device)
    zeros = torch.zeros_like(grid)
    ones = torch.ones_like(grid)
    
    # Points
    pts = [
        torch.stack([grid, zeros], dim=1), # Bottom
        torch.stack([grid, ones], dim=1),  # Top
        torch.stack([zeros, grid], dim=1), # Left
        torch.stack([ones, grid], dim=1)   # Right
    ]
    
    # Normal vectors corresponding to the points
    normals = [
        torch.stack([zeros, -ones], dim=1), # Bottom normal (0, -1)
        torch.stack([zeros, ones], dim=1),  # Top normal (0, 1)
        torch.stack([-ones, zeros], dim=1), # Left normal (-1, 0)
        torch.stack([ones, zeros], dim=1)   # Right normal (1, 0)
    ]
    
    return torch.cat(pts, dim=0), torch.cat(normals, dim=0)

In [5]:
# =================================
# Cell 5: Improved Loss Functional
# =================================
def biharmonic_loss(model, x_int, x_bd, n_bd, bc_weight=10000.0):
    # --- 1. Interior Loss (Same as before) ---
    x_int.requires_grad_()
    u = model(x_int)

    grad_u = torch.autograd.grad(u, x_int, grad_outputs=torch.ones_like(u),
                                create_graph=True, retain_graph=True)[0]

    lap_u = torch.zeros_like(u.squeeze())
    for i in range(2):
        grad_ui = torch.autograd.grad(grad_u[:, i], x_int,
                                     grad_outputs=torch.ones_like(grad_u[:, i]),
                                     create_graph=True, retain_graph=True)[0][:, i]
        lap_u += grad_ui

    grad_lap = torch.autograd.grad(lap_u, x_int,
                                  grad_outputs=torch.ones_like(lap_u),
                                  create_graph=True, retain_graph=True)[0]

    biharmonic = torch.zeros_like(lap_u)
    for i in range(2):
        biharmonic += torch.autograd.grad(grad_lap[:, i], x_int,
                                        grad_outputs=torch.ones_like(grad_lap[:, i]),
                                        create_graph=True)[0][:, i]

    f = rhs_f(x_int)
    pde_loss = ((biharmonic - f)**2).mean()

    # --- 2. Dirichlet Boundary Loss (u = g1) ---
    # Important: Enable gradients for boundary points too!
    x_bd.requires_grad_() 
    u_bd = model(x_bd)
    bc1_loss = ((u_bd.squeeze() - dirichlet_bc(x_bd))**2).mean()
    
    # --- 3. Neumann Boundary Loss (du/dn = g2) ---
    # Calculate gradient at boundary
    grad_u_bd = torch.autograd.grad(u_bd, x_bd, grad_outputs=torch.ones_like(u_bd),
                                   create_graph=True, retain_graph=True)[0]
    
    # Calculate normal derivative: (grad_u . n)
    du_dn_pred = grad_u_bd[:, 0] * n_bd[:, 0] + grad_u_bd[:, 1] * n_bd[:, 1]
    
    # Calculate target
    du_dn_target = neumann_bc(x_bd, n_bd)
    
    bc2_loss = ((du_dn_pred - du_dn_target)**2).mean()

    # Combine losses
    total_loss = pde_loss + bc_weight * (bc1_loss + bc2_loss)
    return total_loss

In [ ]:
# =============================
# Cell 6: Training Configuration
# =============================
epochs = 10000
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                      factor=0.5, patience=500)
losses = []
start_time = time.time()

print("Starting training...")
for epoch in range(epochs):
    model.train()
    x_int, (x_bd, n_bd) = get_interior(N_int), get_boundary(N_bd)
    loss = biharmonic_loss(model, x_int, x_bd, n_bd, bc_weight=10000.0)

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step(loss)

    losses.append(loss.item())

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss {loss.item():.6e}, LR: {scheduler.optimizer.param_groups[0]['lr']:.2e}")

# Final LBFGS optimization
print("Starting LBFGS fine-tuning...")
lbfgs_optimizer = torch.optim.LBFGS(model.parameters(), lr=0.1, max_iter=1000,
                                   tolerance_grad=1e-8, tolerance_change=1e-10,
                                   history_size=100)

def closure():
    lbfgs_optimizer.zero_grad()
    x_int, x_bd = get_interior(N_int), get_boundary(N_bd)
    loss = biharmonic_loss(model, x_int, x_bd, bc_weight=10000.0)
    loss.backward()
    return loss

lbfgs_optimizer.step(closure)

Starting training...


In [ ]:
# ===================
# Cell 7: Save Output
# ===================
torch.save(model.state_dict(), "improved_biharmonic_model_q1.pt")

In [ ]:
# =============================
# Cell 8: Print Architecture, Stats
# =============================
train_time = time.time() - start_time
print(f"\n=== TRAINING SUMMARY ===")
print(f"Training time: {train_time:.2f} seconds")
print("Neural Network Architecture:")
print(model)
n_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {n_params}")
print(f"Non-zero parameters: {sum(torch.count_nonzero(p).item() for p in model.parameters())}")
print(f"Adam initial lr: 1e-3, LBFGS steps: 1000")
print(f"Interior points: {N_int}, Boundary points: {N_bd}")
print(f"Boundary penalty weight: {10000.0}")
print(f"Final training loss: {losses[-1]:.6e}")


In [ ]:
# ===================================
# Cell 9: Plot Solutions and Error
# ===================================
print("\nGenerating plots...")
gsize = 61
xg = np.linspace(0, 1, gsize)
X, Y = np.meshgrid(xg, xg)
grid_points = torch.tensor(np.stack([X.ravel(), Y.ravel()], axis=-1),
                          device=device, dtype=torch.float64)

model.eval()
with torch.no_grad():
    u_nn = model(grid_points).cpu().numpy().reshape(gsize, gsize)
    u_gt = exact_solution(grid_points).cpu().numpy().reshape(gsize, gsize)
    err = np.abs(u_nn - u_gt)

# Neural Network Solution
plt.figure(figsize=(15, 5))

plt.subplot(131)
plt.contourf(X, Y, u_nn, levels=50, cmap='jet')
plt.colorbar()
plt.title('Neural Network Solution')
plt.xlabel('x')
plt.ylabel('y')

# Exact Solution
plt.subplot(132)
plt.contourf(X, Y, u_gt, levels=50, cmap='jet')
plt.colorbar()
plt.title('Exact Solution')
plt.xlabel('x')
plt.ylabel('y')

# Absolute Error
plt.subplot(133)
err_plot = plt.contourf(X, Y, err, levels=50, cmap='hot')
plt.colorbar(err_plot)
plt.title('Absolute Error')
plt.xlabel('x')
plt.ylabel('y')

plt.tight_layout()
plt.savefig("solutions_comparison_q1.png", dpi=300, bbox_inches='tight')
plt.show()

# 3D Plots
fig = plt.figure(figsize=(15, 5))

ax1 = fig.add_subplot(131, projection='3d')
ax1.plot_surface(X, Y, u_nn, cmap='jet', alpha=0.8)
ax1.set_title('NN Solution (3D)')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('u(x,y)')

ax2 = fig.add_subplot(132, projection='3d')
ax2.plot_surface(X, Y, u_gt, cmap='jet', alpha=0.8)
ax2.set_title('Exact Solution (3D)')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('u(x,y)')

ax3 = fig.add_subplot(133, projection='3d')
ax3.plot_surface(X, Y, err, cmap='hot', alpha=0.8)
ax3.set_title('Absolute Error (3D)')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('Error')

plt.tight_layout()
plt.savefig("3d_solutions_q1.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ==================================
# Cell 10: Error Metrics Calculation
# ==================================
print("\nComputing error metrics...")
grid_points.requires_grad = True

# Compute solutions and gradients
u_gt_val = exact_solution(grid_points)
u_nn_val = model(grid_points)

# First derivatives
u_gt_grad = torch.autograd.grad(u_gt_val, grid_points, torch.ones_like(u_gt_val),
                               create_graph=True)[0]
u_nn_grad = torch.autograd.grad(u_nn_val, grid_points, torch.ones_like(u_nn_val),
                               create_graph=True)[0]

# Second derivatives
H_gt = []
H_nn = []
for i in range(2):
    H_gt_i = torch.autograd.grad(u_gt_grad[:, i], grid_points,
                                torch.ones_like(u_gt_grad[:, i]),
                                create_graph=True)[0][:, i]
    H_nn_i = torch.autograd.grad(u_nn_grad[:, i], grid_points,
                                torch.ones_like(u_nn_grad[:, i]),
                                create_graph=True)[0][:, i]
    H_gt.append(H_gt_i.detach().cpu().numpy())
    H_nn.append(H_nn_i.detach().cpu().numpy())

# Convert to numpy for error computation
u_gt_np = u_gt_val.detach().cpu().numpy().ravel()
u_nn_np = u_nn_val.detach().cpu().numpy().ravel()
u_gt_grad_np = u_gt_grad.detach().cpu().numpy()
u_nn_grad_np = u_nn_grad.detach().cpu().numpy()

# Error computation functions
def L2_norm(u):
    return np.sqrt(np.mean(u**2))

def H1_norm(u, grad):
    return np.sqrt(np.mean(u**2) + np.mean(grad[:,0]**2 + grad[:,1]**2))

def H2_norm(u, grad, H):
    return np.sqrt(np.mean(u**2) + np.mean(grad[:,0]**2 + grad[:,1]**2) +
                  np.mean(H[0]**2 + H[1]**2))

# Compute errors
L2_err = L2_norm(u_gt_np - u_nn_np)
H1_err = H1_norm(u_gt_np - u_nn_np, u_gt_grad_np - u_nn_grad_np)
H2_err = H2_norm(u_gt_np - u_nn_np, u_gt_grad_np - u_nn_grad_np,
                [H_gt[0]-H_nn[0], H_gt[1]-H_nn[1]])

# Relative errors
L2_norm_gt = L2_norm(u_gt_np)
H1_norm_gt = H1_norm(u_gt_np, u_gt_grad_np)
H2_norm_gt = H2_norm(u_gt_np, u_gt_grad_np, H_gt)

rel_L2_err = L2_err / L2_norm_gt
rel_H1_err = H1_err / H1_norm_gt
rel_H2_err = H2_err / H2_norm_gt

print("\n=== ERROR METRICS ===")
print(f"L2 Error: {L2_err:.3e}")
print(f"Relative L2 Error: {rel_L2_err:.3e}")
print(f"H1 Error: {H1_err:.3e}")
print(f"Relative H1 Error: {rel_H1_err:.3e}")
print(f"H2 Error: {H2_err:.3e}")
print(f"Relative H2 Error: {rel_H2_err:.3e}")

# Check if requirements are met
print("\n=== REQUIREMENTS CHECK ===")
print(f"Final loss ~1e-5: {'✓' if losses[-1] < 1e-4 else '✗'} (Current: {losses[-1]:.2e})")
print(f"L2 error ~1e-3: {'✓' if L2_err < 2e-3 else '✗'} (Current: {L2_err:.2e})")
print(f"H1 error ~1e-3: {'✓' if H1_err < 2e-3 else '✗'} (Current: {H1_err:.2e})")
print(f"H2 error ~1e-3: {'✓' if H2_err < 2e-3 else '✗'} (Current: {H2_err:.2e})")

In [ ]:
# ===================================
# Cell 11: Plot Loss History
# ===================================
plt.figure(figsize=(10, 6))
plt.semilogy(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss History (Log Scale)')
plt.grid(True, alpha=0.3)
plt.savefig("loss_history_q1.png", dpi=300, bbox_inches='tight')
plt.show()

# Save loss history
with open("loss_history_q1.pkl", "wb") as f:
    pickle.dump(losses, f)

print(f"\nResults saved:")
print("- improved_biharmonic_model.pt: Trained model weights")
print("- solutions_comparison.png: 2D comparison plot")
print("- 3d_solutions.png: 3D surface plots")
print("- loss_history.png: Training dynamics")
print("- loss_history.pkl: Loss history data")